In [1]:
import pandas as pd
import numpy as np

In [2]:
filepath = '/kaggle/input/open-shopee-code-league-logistic/delivery_orders_march.csv'

WORKDAYS = '1111110'
HOLIDAYS = ['2020-03-08','2020-03-25', '2020-03-30', '2020-03-31']

GMT8_OFFSET = 3600 * 8
DURATION_1DAY = 3600 * 24

def mat_to_dict(mat):
    n = len(mat)
    return {i*n+j: mat[i][j] for i in range(n) for j in range(n)}

sla_matrix_1st_attempt = [
    [3, 5, 7, 7],
    [5, 5, 7, 7],
    [7, 7, 7, 7],
    [7, 7, 7, 7],
]
sla_matrix_2nd_attempt = [
    [3, 3, 3, 3],
    [3, 3, 3, 3],
    [3, 3, 3, 3],
    [3, 3, 3, 3],
]
locations = ["Metro Manila", "Luzon", "Visayas", "Mindanao"]
locations = [loc.lower() for loc in locations]
min_length = min(map(len, locations))
trunc_location_to_index = {loc[-min_length:]: i for i, loc in enumerate(locations)}

In [3]:
%%time
dtype = {
    'orderid': np.int64,
    'pick': np.int64,
    '1st_deliver_attempt': np.int64,
    '2nd_deliver_attempt': np.float64,
    'buyeraddress': np.object,
    'selleraddress': np.object,
}
df = pd.read_csv(filepath, dtype=dtype)

CPU times: user 10.9 s, sys: 2.44 s, total: 13.4 s
Wall time: 13.4 s


In [4]:
%%time
# convert address to index
df['buyeraddress'] = df['buyeraddress'].apply(lambda s: s[-min_length:].lower()).map(trunc_location_to_index)
df['selleraddress'] = df['selleraddress'].apply(lambda s: s[-min_length:].lower()).map(trunc_location_to_index)

CPU times: user 4.78 s, sys: 440 ms, total: 5.22 s
Wall time: 5.22 s


In [5]:
%%time
# convert unix datetime(seconds)stamps to unix datetime(date)stamps
dt_columns = ['pick', '1st_deliver_attempt', '2nd_deliver_attempt']
df[dt_columns[-1]] = df['2nd_deliver_attempt'].fillna(0).astype(np.int64)
df[dt_columns] = (df[dt_columns] + GMT8_OFFSET) // DURATION_1DAY

CPU times: user 332 ms, sys: 290 ms, total: 622 ms
Wall time: 588 ms


In [6]:
%%time
# compute number of working days between time intervals
t1 = df['pick'].values.astype('datetime64[D]')
t2 = df['1st_deliver_attempt'].values.astype('datetime64[D]')
t3 = df['2nd_deliver_attempt'].values.astype('datetime64[D]')
df['num_days1'] = np.busday_count(t1, t2, weekmask=WORKDAYS, holidays=HOLIDAYS)
df['num_days2'] = np.busday_count(t2, t3, weekmask=WORKDAYS, holidays=HOLIDAYS)

CPU times: user 214 ms, sys: 22 ms, total: 236 ms
Wall time: 235 ms


In [7]:
%%time
# compute sla based on addresses
to_from = df['buyeraddress']*4 + df['selleraddress']
df['sla1'] = to_from.map(mat_to_dict(sla_matrix_1st_attempt))
df['sla2'] = to_from.map(mat_to_dict(sla_matrix_2nd_attempt))

CPU times: user 85.4 ms, sys: 30.6 ms, total: 116 ms
Wall time: 90.4 ms


In [8]:
%%time
# compute if deliver is late
df['is_late'] = (df['num_days1'] > df['sla1']) | (df['num_days2'] > df['sla2'])
df['is_late'] = df['is_late'].astype(int)

CPU times: user 19.9 ms, sys: 7.81 ms, total: 27.7 ms
Wall time: 27.1 ms


In [9]:
%%time
# write to file
df[['orderid', 'is_late']].to_csv('submission.csv', index=False)

CPU times: user 7.63 s, sys: 400 ms, total: 8.03 s
Wall time: 8.18 s
